# Load Data

In [ ]:
!pip install contractions
!pip install detoxify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
import itertools
import nltk
import numpy as np
import pandas as pd
import re
import string
import contractions

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from detoxify import Detoxify

## Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Original Data


In [ ]:
chunk_size = 100000
combined_data = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the first file
for chunk in pd.read_csv('/content/drive/My Drive/DSA4264 Group Project/Problem 2 Dataset/Reddit-Threads_2020-2021.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  combined_data = pd.concat([combined_data, chunk], ignore_index=True)

# Read and concatenate chunks from the second file
for chunk in pd.read_csv('/content/drive/My Drive/DSA4264 Group Project/Problem 2 Dataset/Reddit-Threads_2022-2023.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  combined_data = pd.concat([combined_data, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {combined_data.shape[0]}")

Total rows in combined data: 4932729


In [ ]:
df = pd.DataFrame(combined_data)

# Data Cleaning

In [ ]:
# Initial shape of dataset
df.shape

(4932729, 9)

In [ ]:
df.describe()

,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation
count,4932675,4932728,4932728,4932728,4932728,4932728,4932728,4932728,4932728
unique,4338648,4787397,105923,4932728,107787,2274423,4932728,3,48
top,[deleted],2020-07-10 19:36:40,[deleted],/r/singapore/comments/gjjem5/covid19_8663_busi...,t3_homxdq,t3_homxdq,fqljinp,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
freq,333164,7,540566,1,10905,5067,1,4444276,1333328


In [ ]:
# Remove rows where comments have been deleted or removed
clean_df = df[df.text != '[deleted]']
clean_df = clean_df[clean_df.text != '[removed]']

# Check for new number of rows and columns
print(clean_df.shape)

(4504384, 9)


In [ ]:
# Check for comments generated by bots
userbots = clean_df[clean_df["username"].str.contains("bot",na=False)]["username"].value_counts()
print(userbots[userbots>20])
print("--------------------------------------------------")
textbots = clean_df[clean_df["text"].str.contains(" bot ",na=False)]["username"].value_counts()
print(textbots[textbots>20])

username
sneakpeek_bot           16431
botsland                 1644
botakchek                1560
SG_wormsbot              1314
wolf-bot                  853
kopibot                   359
bot-amos-counter          343
botoxilum                 339
furby_bot                 314
dontbother111             272
Dankobot                  261
robotguy99                250
sneakpeekbot              243
zombotplus                202
SnooRobots555222          192
kaypohbot                 119
wikipedia_text_bot         98
bigbottlequorn             94
OnlineRobot                92
Xepobot                    81
Tail4mbottllle             71
twitterInfo_bot            65
comfort_bot_1962           64
savevideobot               64
alphabet_order_bot         62
haikusbot                  57
spacerobots5               54
botabolanpa                53
lobotomized_hamster        52
Ramboteo89                 51
BonneybotPG                47
wikipedia_answer_bot       46
zetbotz                    45
O

In [ ]:
# Check the comments made by users that have "bot" in their usernames
for idx, name in enumerate(userbots[userbots>20].index.tolist()):
  print('Username :', name)
  print(clean_df[clean_df['username']==name]["text"][1:10])

Username : sneakpeek_bot
571     &gt; # S'pore customer shows up to 61% price d...
593     &gt; # 25-year-old man to be charged with thro...
792     &gt; # Forum: Health Ministry will study New Z...
1095    &gt; # 12 new locations visited by Covid-19 ca...
1796    &gt; # A story undone: Yale-NUS and future of ...
1945    &gt; # First reported COVID-19 case of KTV clu...
1989    &gt; # Teen drug suspect who died after fall f...
2008    &gt; # Schools can be flexible for students wi...
2084    &gt; # Police investigating murder at Punggol ...
Name: text, dtype: object
Username : botsland
12532    &gt;Get rid of the useless folk in society onc...
16359    The winner takes all system exist in many diff...
18254    &gt;if we had a rule of 8 after a 66% vax rate...
18503    Anyway, the key point is that the Myanmar/ eth...
25872    This tuding issue was raised and discussed bef...
30217    So are you implying that the world is prolongi...
32823    Austria disagrees with you\n\nEdit:\n\n"Unde

In [ ]:
# Check the comments made by users that have "bot" in their comments
for idx, name in enumerate(textbots[textbots>20].index.tolist()):
  print('Username :', name)
  print(clean_df[clean_df['username']==name]["text"][1:10])

Username : SG_wormsbot
2953195    Title: Singapore invokes fake news law for Aus...
2956429    Title: Singapore identity strong enough to wit...
2959738    Original Title: 'I'm so traumatised': Woman mo...
2959803    #Original Title: Police to meet residents each...
2961341    Title: Many S'poreans support longer minimum o...
2961847    #Original Title: Four ministers to speak on Ri...
2962642    #Original Title: Hyundai’s Singapore plant ope...
2965291    #Original Title: Prison inmates to receive tra...
2965325    #Original Title: 'Based on honour system': Bar...
Name: text, dtype: object
Username : AmputatorBot
13475    It looks like you shared an AMP link. These sh...
15960    It looks like you shared an AMP link. These sh...
16999    It looks like you shared an AMP link. These sh...
21407    It looks like you shared an AMP link. These sh...
23153    It looks like you shared an AMP link. These sh...
27007    It looks like OP posted an AMP link. These sho...
32514    It looks like y

In [ ]:
# Change comments generated by bots to NA values, which will be removed later on
bots = ["sneakpeek_bot","SG_wormsbot","bot-amos-counter","sneakpeekbot","wikipedia_text_bot","alphabet_order_bot",
        "wikipedia_answer_bot","sub_doesnt_exist_bot","remindditbot","happy-cake-day-bot-","useles-converter-bot",
        "same_post_bot","AmputatorBot","VredditDownloader","B0tRank","AutoModerator"]

clean_df.loc[clean_df['username'].isin(bots), 'text'] = np.nan

In [ ]:
# Remove rows with NA values in the 'text' column or those with NA values in the 'timestamp' column
clean_df = clean_df.dropna(subset=['text', 'timestamp'])

# Check for new number of rows and columns
print(clean_df.shape)

(4474070, 9)


In [ ]:
clean_df.describe()

,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation
count,4474070,4474070,4474070,4474070,4474070,4474070,4474070,4474070,4474070
unique,4319827,4355462,105906,4474070,100681,2140153,4474070,3,41
top,Yes,2020-07-10 19:36:40,[deleted],/r/singapore/comments/gjjem5/covid19_8663_busi...,t3_homxdq,t3_homxdq,fqljinp,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
freq,2612,7,112260,1,9803,4566,1,4018132,1325307


Removing duplicates

In [ ]:
# Remove duplicated texts with same timestamp, username
df_unique = clean_df.drop_duplicates(subset=['username', 'text', 'timestamp'], keep='first')
df_unique.shape

(4473811, 9)

# Data Transformation

In [ ]:
# Remove moderation and link_id columns
df_unique = df_unique.drop(columns=['moderation','link_id'])

In [ ]:
# Extract topic from link column
def extract_topic(row):
    link = row['link']
    id_value = row['id']

    # Split the link by '/' and remove empty elements
    parts = [part for part in link.split('/') if part]

    # Find the position of the id and extract the element before it (the topic)
    if id_value in parts:
        id_index = parts.index(id_value)
        if id_index > 0:
            return parts[id_index - 1].replace('_', ' ')  # The topic is just before the id
    return None  # Return None if id not found

# Apply function to extract the topic
df_unique['topic'] = df_unique.apply(extract_topic, axis=1)


# Display df with extracted topics
print(df_unique[["link", "topic"]])

                                                      link  \
0        /r/singapore/comments/gjjem5/covid19_8663_busi...   
1        /r/singapore/comments/f15aks/did_i_just_get_sc...   
3        /r/singapore/comments/l28wfr/rsingapore_random...   
4        /r/singapore/comments/fp5hgu/pcf_cluster_anoth...   
5        /r/singapore/comments/nem5j0/package_from_sing...   
...                                                    ...   
4932724  /r/singapore/comments/ux28gq/rsingapore_random...   
4932725  /r/singapore/comments/xqprla/rsingapore_random...   
4932726  /r/singapore/comments/1235s44/eli5_why_do_the_...   
4932727  /r/singapore/comments/130dj90/latest_property_...   
4932728  /r/singaporehappenings/comments/13iyh0x/uncle_...   

                                                     topic  
0          covid19 8663 businesses in spore closed down in  
1                                   did i just get scammed  
3         rsingapore random discussion and small questions  
4          

In [ ]:
# Convert the 'timestamp' column to datetime format
df_unique['timestamp'] = pd.to_datetime(df_unique['timestamp'])

# Create separate 'date' and 'time' columns
df_unique['date'] = df_unique['timestamp'].dt.date      # Extract date part
df_unique['time'] = df_unique['timestamp'].dt.time

df_unique

,text,timestamp,username,link,parent_id,id,subreddit_id,topic,date,time
0,STI chiong ah,2020-05-14 12:35:30,iamabear1,/r/singapore/comments/gjjem5/covid19_8663_busi...,t3_gjjem5,fqljinp,t5_2qh8c,covid19 8663 businesses in spore closed down in,2020-05-14,12:35:30
1,Look on the bright side - you'll never make th...,2020-02-09 17:23:24,lkc159,/r/singapore/comments/f15aks/did_i_just_get_sc...,t3_f15aks,fh3hl0g,t5_2qh8c,did i just get scammed,2020-02-09,17:23:24
3,sounds q fucked up if no concern for each othe...,2021-01-22 14:22:42,[deleted],/r/singapore/comments/l28wfr/rsingapore_random...,t1_gk6fcys,gk6gc0y,t5_2qh8c,rsingapore random discussion and small questions,2021-01-22,14:22:42
4,Chinese media reported a while ago: https://ww...,2020-03-26 04:51:22,localinfluenza,/r/singapore/comments/fp5hgu/pcf_cluster_anoth...,t3_fp5hgu,flj42mf,t5_2qh8c,pcf cluster another teacher diagnosed with,2020-03-26,04:51:22
5,Takes forever. I would pay extra for DHL next ...,2021-05-17 17:54:57,flippingnoob,/r/singapore/comments/nem5j0/package_from_sing...,t3_nem5j0,gygzhjw,t5_2qh8c,package from singpost to us,2021-05-17,17:54:57
...,...,...,...,...,...,...,...,...,...,...
4932724,Its fuckin hot rn,2022-05-25 04:22:50,Mahsunon,/r/singapore/comments/ux28gq/rsingapore_random...,t1_i9waspv,i9wb51x,t5_2qh8c,rsingapore random discussion and small questions,2022-05-25,04:22:50
4932725,"hey, sending hugs your way &lt;3",2022-09-29 14:05:32,_0_o,/r/singapore/comments/xqprla/rsingapore_random...,t1_iqcvjsd,iqd9twj,t5_2qh8c,rsingapore random discussion and small questions,2022-09-29,14:05:32
4932726,Depends in the speed they got caught doing. In...,2023-03-27 01:00:34,dodgethis_sg,/r/singapore/comments/1235s44/eli5_why_do_the_...,t3_1235s44,jdtcn3x,t5_2qh8c,eli5 why do the traffic police let offenders go,2023-03-27,01:00:34
4932727,Maybe I should have typed ‘doesn’t change too ...,2023-04-29 04:43:31,ShadeX8,/r/singapore/comments/130dj90/latest_property_...,t1_ji5arfa,ji5d2nc,t5_2qh8c,latest property cooling measures unlikely to ease,2023-04-29,04:43:31


In [ ]:
# Remove link and timestamp columns
df_unique = df_unique.drop(columns=['link','timestamp'])

In [ ]:
# Check updated dataframe
df_unique

,text,username,parent_id,id,subreddit_id,topic,date,time
0,STI chiong ah,iamabear1,t3_gjjem5,fqljinp,t5_2qh8c,covid19 8663 businesses in spore closed down in,2020-05-14,12:35:30
1,Look on the bright side - you'll never make th...,lkc159,t3_f15aks,fh3hl0g,t5_2qh8c,did i just get scammed,2020-02-09,17:23:24
3,sounds q fucked up if no concern for each othe...,[deleted],t1_gk6fcys,gk6gc0y,t5_2qh8c,rsingapore random discussion and small questions,2021-01-22,14:22:42
4,Chinese media reported a while ago: https://ww...,localinfluenza,t3_fp5hgu,flj42mf,t5_2qh8c,pcf cluster another teacher diagnosed with,2020-03-26,04:51:22
5,Takes forever. I would pay extra for DHL next ...,flippingnoob,t3_nem5j0,gygzhjw,t5_2qh8c,package from singpost to us,2021-05-17,17:54:57
...,...,...,...,...,...,...,...,...
4932724,Its fuckin hot rn,Mahsunon,t1_i9waspv,i9wb51x,t5_2qh8c,rsingapore random discussion and small questions,2022-05-25,04:22:50
4932725,"hey, sending hugs your way &lt;3",_0_o,t1_iqcvjsd,iqd9twj,t5_2qh8c,rsingapore random discussion and small questions,2022-09-29,14:05:32
4932726,Depends in the speed they got caught doing. In...,dodgethis_sg,t3_1235s44,jdtcn3x,t5_2qh8c,eli5 why do the traffic police let offenders go,2023-03-27,01:00:34
4932727,Maybe I should have typed ‘doesn’t change too ...,ShadeX8,t1_ji5arfa,ji5d2nc,t5_2qh8c,latest property cooling measures unlikely to ease,2023-04-29,04:43:31


## Saves a csv file

In [ ]:
df_unique.to_csv("reddit_data_temp.csv", index=False)

## Handling Language

In [ ]:
chunk_size = 100000
df_unique = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the file
for chunk in pd.read_csv('/content/drive/My Drive/DSA4264 Group Project/dsa4264_datasets/reddit_data_temp.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  df_unique = pd.concat([df_unique, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {df_unique.shape[0]}")

Total rows in combined data: 4473811


In [ ]:
df_unique.head()

,text,username,parent_id,id,subreddit_id,topic,date,time
0,STI chiong ah,iamabear1,t3_gjjem5,fqljinp,t5_2qh8c,covid19 8663 businesses in spore closed down in,2020-05-14,12:35:30
1,Look on the bright side - you'll never make th...,lkc159,t3_f15aks,fh3hl0g,t5_2qh8c,did i just get scammed,2020-02-09,17:23:24
2,sounds q fucked up if no concern for each othe...,[deleted],t1_gk6fcys,gk6gc0y,t5_2qh8c,rsingapore random discussion and small questions,2021-01-22,14:22:42
3,Chinese media reported a while ago: https://ww...,localinfluenza,t3_fp5hgu,flj42mf,t5_2qh8c,pcf cluster another teacher diagnosed with,2020-03-26,04:51:22
4,Takes forever. I would pay extra for DHL next ...,flippingnoob,t3_nem5j0,gygzhjw,t5_2qh8c,package from singpost to us,2021-05-17,17:54:57


In [ ]:
def clean_text(text):
  # Convert to lowercase
  text = text.lower()
  # Remove HTML tags
  text = BeautifulSoup(text, "html.parser").get_text()
  # Remove URLs
  text = re.sub(r'http\S+|www\S+', '', text)
  # Standardize words (remove repeated letters, e.g., happpppy -> happy)
  text = re.sub(r'(.)\1+', r'\1\1', text)  # Keeps only up to 2 of the same letter
  # Remove extra whitespaces
  text = text.strip()

  return text

# Apply cleaning function to 'text' column
df_unique['cleaned_text'] = df_unique['text'].apply(clean_text)

<ipython-input-24-3c5190677e77>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-24-3c5190677e77>:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
# Create list of Singlish stopwords
singlish_stopwords = ["la","lah","leh","ley","lo","lor","lorh","ba","bah","barh","sia","siah","sial","siol","eh","ey",
                      "ah","ahh","arh","aiyo","aiyoh","aiya","aiyah","hor","horh","ma","meh","marh","orh","wah","wa",
                      "chey","wor","worh","liao","de","lahz","nia","lahhh","bo","lahh","hais","haish","haiz","aish",
                      "erm","ermm","er","err","urm","urmm","uhh","uh","um","umm","hm","hmm"]

# Create function to remove singlish stopwords from text
def remove_singlish_stopwords(txt):
  # Use regex to remove each stopword surrounded by word boundaries
  pattern = r'\b(' + '|'.join(re.escape(word) for word in singlish_stopwords) + r')\b'
  cleaned_txt = re.sub(pattern, '', txt)

  # Remove extra whitespace created by the replacement
  return ' '.join(cleaned_txt.split())

# Remove singlish stopwords from 'cleaned_text' column
df_unique['cleaned_text'] = df_unique['cleaned_text'].apply(remove_singlish_stopwords)

In [ ]:
# Mapping abbreviations
abbreviation_df = pd.read_excel('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/abbreviations_and_slangs.xlsx')

# Create a dictionary for mapping abbreviations to full forms
abbreviation_dict = pd.Series(abbreviation_df.Text.values, index=abbreviation_df.Abbreviations).to_dict()

# Create function to replace abbreviations in text
def replace_abbreviations(text, mapping):
    words = text.split()
    replaced_words = [mapping.get(word, word) for word in words]
    return ' '.join(replaced_words)

# Replace abbreviations in the 'cleaned_text' column
df_unique['cleaned_text'] = df_unique['cleaned_text'].apply(lambda x: replace_abbreviations(x, abbreviation_dict))

print(df_unique)

                                                      text  \
0                                            STI chiong ah   
1        Look on the bright side - you'll never make th...   
2        sounds q fucked up if no concern for each othe...   
3        Chinese media reported a while ago: https://ww...   
4        Takes forever. I would pay extra for DHL next ...   
...                                                    ...   
4473806                                  Its fuckin hot rn   
4473807                   hey, sending hugs your way &lt;3   
4473808  Depends in the speed they got caught doing. In...   
4473809  Maybe I should have typed ‘doesn’t change too ...   
4473810                   uncle buay tahan by the heatwave   

                     username   parent_id       id subreddit_id  \
0                   iamabear1   t3_gjjem5  fqljinp     t5_2qh8c   
1                      lkc159   t3_f15aks  fh3hl0g     t5_2qh8c   
2                   [deleted]  t1_gk6fcys  gk6gc0y    

In [ ]:
# Create function to clean contractions
def clean_contractions(txt):
    if txt.endswith("."):
        txt = txt[:-1]
        results = []
        for word in txt.split():
            results.append(contractions.fix(word))
        results = ' '.join(results)
        results += "."
    else:
        results = []
        for word in txt.split():
            results.append(contractions.fix(word))
        results = ' '.join(results)
    return results

# Clean contractions in 'cleaned_text' column
df_unique["cleaned_text"] = df_unique["cleaned_text"].apply(clean_contractions)

In [ ]:
# Remove emojis and chinese characters
def remove_emojis(data):
  emoj = re.compile("["
    u"\U0001F600-\U0001F64F"  # Emoticons
    u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # Transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
    u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    u"\U00002500-\U00002BEF"  # Chinese characters
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\U00002702-\U000027B0"  # Dingbats
    u"\u2600-\u26FF"          # Miscellaneous Symbols (e.g., ☀, ☁)
    u"\u2700-\u27BF"          # Dingbats (e.g., ✂, ✉)
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"
    u"\u3030"
                  "]+", re.UNICODE)
  return re.sub(emoj, '', data)

df_unique['cleaned_text'] = df_unique['cleaned_text'].apply(remove_emojis)

In [ ]:
# Check final dataframe
print(df_unique)

                                                      text  \
0                                            STI chiong ah   
1        Look on the bright side - you'll never make th...   
2        sounds q fucked up if no concern for each othe...   
3        Chinese media reported a while ago: https://ww...   
4        Takes forever. I would pay extra for DHL next ...   
...                                                    ...   
4473806                                  Its fuckin hot rn   
4473807                   hey, sending hugs your way &lt;3   
4473808  Depends in the speed they got caught doing. In...   
4473809  Maybe I should have typed ‘doesn’t change too ...   
4473810                   uncle buay tahan by the heatwave   

                     username   parent_id       id subreddit_id  \
0                   iamabear1   t3_gjjem5  fqljinp     t5_2qh8c   
1                      lkc159   t3_f15aks  fh3hl0g     t5_2qh8c   
2                   [deleted]  t1_gk6fcys  gk6gc0y    

## Clean the dataset once more, by removing empty strings in the 'cleaned_text' column

In [ ]:
# Remove leading/trailing whitespace
df_unique['cleaned_text'] = df_unique['cleaned_text'].str.strip()
print(f"Number of rows in the dataset: {df_unique.shape[0]}")

# Remove rows with 'cleaned_text' as empty strings
df_unique = df_unique[df_unique["cleaned_text"]!= '']
print(f"Number of rows in the dataset after removing empty strings: {df_unique.shape[0]}")

Number of rows in the dataset: 4473811
Number of rows in the dataset after removing empty strings: 4452622


## Saves the dataset, which will be used as input for text classification

In [ ]:
df_unique.to_csv("/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_data_llm.csv",index=False)

## Checks that df_unique is properly saved

In [ ]:
chunk_size = 100000
df_unique = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the file
for chunk in pd.read_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_data_llm.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  df_unique = pd.concat([df_unique, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {df_unique.shape[0]}")

df_unique.head()

#Classification

## Preprocess Jigsaw Dataset

### Clean train set

In [ ]:
# Read train data from Kaggle dataset
df_train = pd.read_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/classification/train.csv')

In [ ]:
# Check for NaN values in 'comment_text' column
sum(df_train['comment_text'].isna())

0

In [ ]:
df_cleaned = df_train.copy()

In [ ]:
# Apply text cleaning function to 'comment_text' column
df_cleaned['comment_text'] = df_cleaned['comment_text'].apply(clean_text)

# Remove singilsh stopwords from 'comment_text' column
df_cleaned['comment_text'] = df_cleaned['comment_text'].apply(remove_singlish_stopwords)

# Replace abbreviations in the 'comment_text' column
df_cleaned['comment_text'] = df_cleaned['comment_text'].apply(lambda x: replace_abbreviations(x, abbreviation_dict))

# Cleaning contractions from 'comment_text' column
df_cleaned["comment_text"] = df_cleaned["comment_text"].apply(clean_contractions)

# Remove emojis and chinese characters from the 'comment_text' column
df_cleaned['comment_text'] = df_cleaned['comment_text'].apply(remove_emojis)

<ipython-input-24-3c5190677e77>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-24-3c5190677e77>:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
# Clean the dataset once more, by removing empty strings in the 'comment_text' column
df_cleaned['comment_text'] = df_cleaned['comment_text'].str.strip()  # Remove leading/trailing whitespace
print(f"Number of rows in the dataset: {df_cleaned.shape[0]}")

# Remove rows where 'comment_text' column is an empty string
df_cleaned = df_cleaned[df_cleaned['comment_text']!= '']
print(f"Number of rows in the dataset after removing empty strings: {df_cleaned.shape[0]}")

Number of rows in the dataset: 159571
Number of rows in the dataset after removing empty strings: 159564


In [ ]:
# Save the cleaned training data to csv
df_cleaned.to_csv("/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/classification/train_cleaned.csv",index=False)

### Clean test set

In [ ]:
# Read test set and test labels
test = pd.read_csv("/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/classification/test.csv")
labels = pd.read_csv("/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/classification/test_labels.csv")

In [ ]:
# Merge the two dataframes by id
test_label = pd.merge(test, labels, on='id', how='inner')
print(test_label.head())

                 id                                       comment_text  toxic  \
0  00001cee341fdb12  Yo bitch Ja Rule is more succesful then you'll...     -1   
1  0000247867823ef7  == From RfC == \n\n The title is fine as it is...     -1   
2  00013b17ad220c46  " \n\n == Sources == \n\n * Zawe Ashton on Lap...     -1   
3  00017563c3f7919a  :If you have a look back at the source, the in...     -1   
4  00017695ad8997eb          I don't anonymously edit articles at all.     -1   

   severe_toxic  obscene  threat  insult  identity_hate  
0            -1       -1      -1      -1             -1  
1            -1       -1      -1      -1             -1  
2            -1       -1      -1      -1             -1  
3            -1       -1      -1      -1             -1  
4            -1       -1      -1      -1             -1  


In [ ]:
# Remove rows where any of the test labels = -1
df_filtered = test_label[~(test_label.iloc[:, 2:] == -1).any(axis=1)]

print(df_filtered.head())

                  id                                       comment_text  \
5   0001ea8717f6de06  Thank you for understanding. I think very high...   
7   000247e83dcc1211                   :Dear god this site is horrible.   
11  0002f87b16116a7f  "::: Somebody will invariably try to add Relig...   
13  0003e1cccfd5a40a  " \n\n It says it right there that it IS a typ...   
14  00059ace3e3e9a53  " \n\n == Before adding a new product to the l...   

    toxic  severe_toxic  obscene  threat  insult  identity_hate  
5       0             0        0       0       0              0  
7       0             0        0       0       0              0  
11      0             0        0       0       0              0  
13      0             0        0       0       0              0  
14      0             0        0       0       0              0  


In [ ]:
df_cleaned = df_filtered.copy()

In [ ]:
# Apply cleaning function to 'comment_text' column
df_cleaned['comment_text'] = df_cleaned['comment_text'].apply(clean_text)

# Remove singilsh stopwords from 'comment_text' column
df_cleaned['comment_text'] = df_cleaned['comment_text'].apply(remove_singlish_stopwords)

# Replace abbreviations in the 'comment_text' column
df_cleaned['comment_text'] = df_cleaned['comment_text'].apply(lambda x: replace_abbreviations(x, abbreviation_dict))

# Cleaning contractions from 'comment_text' column
df_cleaned["comment_text"] = df_cleaned["comment_text"].apply(clean_contractions)

# Remove emojis and chinese characters from the 'comment_text' column
df_cleaned['comment_text'] = df_cleaned['comment_text'].apply(remove_emojis)

<ipython-input-24-3c5190677e77>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-24-3c5190677e77>:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
# Clean the dataset once more, by removing empty strings in the 'comment_text' column
df_cleaned['comment_text'] = df_cleaned['comment_text'].str.strip()  # Remove leading/trailing whitespace
print(f"Number of rows in the dataset: {df_cleaned.shape[0]}")

# Remove the rows where 'comment_text' is an empty string
df_cleaned = df_cleaned[df_cleaned['comment_text']!= '']
print(f"Number of rows in the dataset after removing empty strings: {df_cleaned.shape[0]}")

Number of rows in the dataset: 63978
Number of rows in the dataset after removing empty strings: 63969


In [ ]:
# Split the cleaned data to test set and label set
test_cleaned = df_cleaned.iloc[:, [0,1]]
label_cleaned = df_cleaned.iloc[:,[0]].join(df_cleaned.iloc[:,2:])

In [ ]:
print(test_cleaned.head())
print("")
print(label_cleaned.head())

                  id                                       comment_text
5   0001ea8717f6de06  thank you for understanding. i think very high...
7   000247e83dcc1211                   :dear god this site is horrible.
11  0002f87b16116a7f  ":: somebody will invariably try to add religi...
13  0003e1cccfd5a40a  " it says it right there that it is a type. th...
14  00059ace3e3e9a53  " == before adding a new product to the list, ...

                  id  toxic  severe_toxic  obscene  threat  insult  \
5   0001ea8717f6de06      0             0        0       0       0   
7   000247e83dcc1211      0             0        0       0       0   
11  0002f87b16116a7f      0             0        0       0       0   
13  0003e1cccfd5a40a      0             0        0       0       0   
14  00059ace3e3e9a53      0             0        0       0       0   

    identity_hate  
5               0  
7               0  
11              0  
13              0  
14              0  


In [ ]:
# Save both datasets to csv
test_cleaned.to_csv("/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/classification/test_cleaned.csv",index=False)
label_cleaned.to_csv("/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/classification/label_cleaned.csv",index=False)

# Stratified Sampling on Reddit Data

In [ ]:
# Read in data
chunk_size = 100000
df = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the file
for chunk in pd.read_csv('/content/drive/My Drive/DSA4264 Group Project/dsa4264_datasets/reddit_data_llm.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  df = pd.concat([df, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {df.shape[0]}")

Total rows in combined data: 4452621


In [ ]:
# Calculate number of comments in each subreddit
subreddit_count = df['subreddit_id'].value_counts()

# Total number of rows
total = subreddit_count.sum()

# Calculate proportion of each subreddit
t5_2qh8c = subreddit_count.iloc[0]/total
t5_xnx04 = subreddit_count.iloc[1]/total
t5_70s6ew = subreddit_count.iloc[2]/total

print(f't5_2qh8c = {t5_2qh8c}')
print(f't5_xnx04 = {t5_xnx04}')
print(f't5_70s6ew = {t5_70s6ew}')

t5_2qh8c = 0.8982170276787537
t5_xnx04 = 0.09130734459546411
t5_70s6ew = 0.010475627725782186


In [ ]:
# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a new column 'quarter_year' with the format 'YYYYQ#'
df['quarter_year'] = df['date'].dt.to_period('Q').astype(str)

In [ ]:
# Calculate the target size (5% of the original dataset)
target_size = int(0.05 * len(df))

# Calculate the desired number of rows for each quarter
rows_per_quarter = target_size // 16

# Ensure each `subreddit_id` is represented proportionally within each quarter
sampled_data = pd.DataFrame()  # Placeholder for the stratified sample

q = df['quarter_year'].unique().tolist()

for quarter in q:
    # Filter rows for the specific quarter (assuming you have a 'quarter' column)
    quarter_data = df[df['quarter_year'] == quarter]

    # Sample from each 'subreddit_id' within the quarter
    stratified_sample = quarter_data.groupby('subreddit_id', group_keys=False).apply(
        lambda x: x.sample(frac=rows_per_quarter / len(quarter_data)).reset_index(drop=True)
    )

    # Add the stratified sample for this quarter to the final dataset
    sampled_data = pd.concat([sampled_data, stratified_sample])

# Reset index for the final stratified sample
sampled_data = sampled_data.reset_index(drop=True)

<ipython-input-8-778789d5bc5e>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_sample = quarter_data.groupby('subreddit_id', group_keys=False).apply(
<ipython-input-8-778789d5bc5e>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_sample = quarter_data.groupby('subreddit_id', group_keys=False).apply(
<ipython-input-8-778789d5bc5e>:17: DeprecationWarning: DataFrameGroupBy.apply operat

In [ ]:
# Check the results
print(sampled_data.head())

                                                text            username  \
0  They removed Michael Palmer over infidelity bu...             hironyx   
1  So what books are you guys currently reading? ...           Born-Mind   
2  how do you start ordering? cant find it on ios...  BoyfriendSharkDudu   
3  Bodyline drills (planks &amp; variations), hol...           barurutor   
4                           Preparing for nose swab?         rycbarm1234   

    parent_id       id subreddit_id  \
0   t3_hgocs6  fw5iddz     t5_2qh8c   
1   t3_h9pyf2  fv1ugpt     t5_2qh8c   
2   t3_gnouvq  frhe1p8     t5_2qh8c   
3   t3_gcn72r  fpcklpy     t5_2qh8c   
4  t1_fo62oye  fo62pqr     t5_2qh8c   

                                               topic       date      time  \
0  the rsm who first exposed ivan is reaching out to 2020-06-27  09:38:20   
1   rsingapore random discussion and small questions 2020-06-16  20:24:49   
2   the singaporeans helping to save the fb industry 2020-05-22  20:10:13   


In [ ]:
df = sampled_data.copy()

In [ ]:
# Calculate number of comments in each subreddit
subreddit_count = df['subreddit_id'].value_counts()

# Total number of rows
total = subreddit_count.sum()

# Check proportion of each subreddit (same as before sampling)
t5_2qh8c = subreddit_count.iloc[0]/total
t5_xnx04 = subreddit_count.iloc[1]/total
t5_70s6ew = subreddit_count.iloc[2]/total

print(f't5_2qh8c = {t5_2qh8c}')
print(f't5_xnx04 = {t5_xnx04}')
print(f't5_70s6ew = {t5_70s6ew}')

t5_2qh8c = 0.8385297116650121
t5_xnx04 = 0.13556550760703073
t5_70s6ew = 0.02590478072795713


In [ ]:
# Check for equal number of rows for each quarter
print(df['quarter_year'].value_counts())

quarter_year
2020Q2    13914
2020Q1    13914
2021Q1    13914
2021Q2    13914
2021Q3    13914
2020Q3    13914
2021Q4    13914
2020Q4    13914
2023Q3    13914
2022Q1    13914
2022Q4    13914
2022Q3    13914
2023Q1    13914
2022Q2    13914
2023Q4    13914
2023Q2    13913
Name: count, dtype: int64


In [ ]:
# Save sampled data to csv
sampled_data.to_csv("sampled_data.csv",index=False)

# Labelling of Reddit Data

In [ ]:
chunk_size = 100000
reddit_sample = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the file
for chunk in pd.read_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/sampled_data.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  reddit_sample = pd.concat([reddit_sample, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {reddit_sample.shape[0]}")

Total rows in combined data: 222623


## Split into smaller chunks for labelling

In [ ]:
chunk_size = 40000
num_chunks = (len(reddit_sample) // chunk_size) + (1 if len(reddit_sample) % chunk_size > 0 else 0)

# Split and save the chunks
for i in range(num_chunks):
    start_index = i * chunk_size
    end_index = start_index + chunk_size
    chunk = reddit_sample.iloc[start_index:end_index]
    chunk.to_csv(f'reddit_sample_chunk_{i + 1}.csv', index=False)

## Label reddit data

### Label chunk 1

In [ ]:
chunk_size = 100000
reddit_sample_chunk_1 = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the file
for chunk in pd.read_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/reddit_sample_chunk_1.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  reddit_sample_chunk_1 = pd.concat([reddit_sample_chunk_1, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {reddit_sample_chunk_1.shape[0]}")

Total rows in combined data: 40000


In [ ]:
model = 'unbiased'

labels = []
for i in reddit_sample_chunk_1['text']:
    result = Detoxify(model).predict(i)
    labels.append(result)
reddit_sample_chunk_1['label'] = labels
reddit_sample_chunk_1

In [ ]:
reddit_sample_chunk_1.to_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/labelled_reddit_sample_chunk_1.csv', index=False)

### Label chunk 2

In [ ]:
chunk_size = 100000
reddit_sample_chunk_2 = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the file
for chunk in pd.read_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/reddit_sample_chunk_2.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  reddit_sample_chunk_2 = pd.concat([reddit_sample_chunk_2, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {reddit_sample_chunk_2.shape[0]}")

In [ ]:
model = 'unbiased'

labels = []
for i in reddit_sample_chunk_2['text']:
    result = Detoxify(model).predict(i)
    labels.append(result)
reddit_sample_chunk_2['label'] = labels
reddit_sample_chunk_2

In [ ]:
reddit_sample_chunk_2.to_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/labelled_reddit_sample_chunk_2.csv', index=False)

### Label chunk 3

In [ ]:
chunk_size = 100000
reddit_sample_chunk_3 = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the file
for chunk in pd.read_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/reddit_sample_chunk_3.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  reddit_sample_chunk_3 = pd.concat([reddit_sample_chunk_3, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {reddit_sample_chunk_3.shape[0]}")

In [ ]:
model = 'unbiased'

labels = []
for i in reddit_sample_chunk_3['text']:
    result = Detoxify(model).predict(i)
    labels.append(result)
reddit_sample_chunk_3['label'] = labels
reddit_sample_chunk_3

In [ ]:
reddit_sample_chunk_3.to_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/labelled_reddit_sample_chunk_3.csv', index=False)

### Label chunk 4

In [ ]:
chunk_size = 100000
reddit_sample_chunk_4 = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the file
for chunk in pd.read_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/reddit_sample_chunk_4.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  reddit_sample_chunk_4 = pd.concat([reddit_sample_chunk_4, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {reddit_sample_chunk_4.shape[0]}")

Total rows in combined data: 40000


In [ ]:
model = 'unbiased'

labels = []
for i in reddit_sample_chunk_4['text']:
    result = Detoxify(model).predict(i)
    labels.append(result)
reddit_sample_chunk_4['label'] = labels
reddit_sample_chunk_4

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.3-alpha/toxic_debiased-c7548aa0.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_debiased-c7548aa0.ckpt
100%|██████████| 476M/476M [00:05<00:00, 84.6MB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
reddit_sample_chunk_4.to_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/labelled_reddit_sample_chunk_4.csv', index=False)

### Label chunk 5

In [ ]:
chunk_size = 100000
reddit_sample_chunk_5 = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the file
for chunk in pd.read_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/reddit_sample_chunk_5.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  reddit_sample_chunk_5 = pd.concat([reddit_sample_chunk_5, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {reddit_sample_chunk_5.shape[0]}")

Total rows in combined data: 40000


In [ ]:
model = 'unbiased'

labels = []
for i in reddit_sample_chunk_5['text']:
    result = Detoxify(model).predict(i)
    labels.append(result)
reddit_sample_chunk_5['label'] = labels
reddit_sample_chunk_5

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.3-alpha/toxic_debiased-c7548aa0.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_debiased-c7548aa0.ckpt
100%|██████████| 476M/476M [00:05<00:00, 84.4MB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
reddit_sample_chunk_5.to_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/labelled_reddit_sample_chunk_5.csv', index=False)

### Label chunk 6

In [ ]:
chunk_size = 100000
reddit_sample_chunk_6 = pd.DataFrame()  # Initialize an empty DataFrame

# Read and concatenate chunks from the file
for chunk in pd.read_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/reddit_sample_chunk_6.csv', chunksize=chunk_size, engine='python', on_bad_lines='warn', encoding='utf-8'):
  reddit_sample_chunk_6 = pd.concat([reddit_sample_chunk_6, chunk], ignore_index=True)

# Check total number of rows
print(f"Total rows in combined data: {reddit_sample_chunk_6.shape[0]}")

In [ ]:
model = 'unbiased'

labels = []
for i in reddit_sample_chunk_6['text']:
    result = Detoxify(model).predict(i)
    labels.append(result)
reddit_sample_chunk_6['label'] = labels
reddit_sample_chunk_6

In [ ]:
reddit_sample_chunk_6.to_csv('/content/drive/MyDrive/DSA4264 Group Project/dsa4264_datasets/reddit_sample_data/labelled_reddit_sample_chunk_6.csv', index=False)